# Make simple catalogue of CIGALE objects for HiPS

The aim is to make a simplified list of all CIGALE objects so that we can display objects including SEDs over an Aladin Lite map.

In [1]:
import pyvo as vo
import glob
import time
import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt
import random
import yaml

from astropy.coordinates import SkyCoord
from astropy import units as u

from astropy.table import Table, Column, vstack, join

from herschelhelp_internal.utils import inMoc, gen_help_id

from pymoc import MOC

In [2]:
cigale_query = """
SELECT 
    field, 
    help_id, 
    ra, 
    dec, 
    redshift, 
    zspec,
    f_pacs_green,
    f_pacs_red,
    f_spire_250,
    f_spire_350,
    f_spire_500,
    ferr_pacs_green,
    ferr_pacs_red,
    ferr_spire_250,
    ferr_spire_350,
    ferr_spire_500,
    cigale_dustlumin,
    cigale_dustlumin_err,
    cigale_dustlumin_ironly,
    cigale_dustlumin_ironly_err,
    cigale_mstar,
    cigale_mstar_err,
    cigale_sfr,
    cigale_sfr_err,
    flag_optnir_obs,
    flag_optnir_det,
    hp_idx
FROM herschelhelp.main 
WHERE (herschelhelp.main.cigale_dustlumin > 0)"""



# construct a service; I’ve taken the URL from TOPCAT’s
# TAP service browser # ("Selected TAP Service" near the
# foot of the dialog)
service = vo.dal.TAPService(
          "https://herschel-vos.phys.sussex.ac.uk/__system__/tap/run/tap")

In [3]:
job = service.submit_job(cigale_query)
job.run()
job_url = job.url
job_result = vo.dal.tap.AsyncTAPJob(job_url)
start_time = time.time()
wait = 10
while job.phase == 'EXECUTING':
    print('Job still running after {} seconds.'.format(round(time.time() - start_time)))
    time.sleep(wait) #wait ten seconds and try again
    wait = 2*wait

print(job.phase)
result = job_result.fetch_result()

Job still running after 1 seconds.
Job still running after 11 seconds.
Job still running after 32 seconds.
Job still running after 73 seconds.
Job still running after 153 seconds.
Job still running after 314 seconds.
COMPLETED


In [4]:
cigale_objects = result.table
cigale_objects[:10].show_in_notebook()

idx,field,help_id,ra,dec,redshift,zspec,f_pacs_green,f_pacs_red,f_spire_250,f_spire_350,f_spire_500,ferr_pacs_green,ferr_pacs_red,ferr_spire_250,ferr_spire_350,ferr_spire_500,cigale_dustlumin,cigale_dustlumin_err,cigale_dustlumin_ironly,cigale_dustlumin_ironly_err,cigale_mstar,cigale_mstar_err,cigale_sfr,cigale_sfr_err,flag_optnir_obs,flag_optnir_det,hp_idx
,,,deg,deg,,,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,W,W,W,W,Msun,Msun,Msun / yr,Msun / yr,,,
0,HATLAS-SGP,HELP_J234040.389-315108.706,355.16828789586,-31.8524183563571,1.0735,--,--,--,2985.9941,15432.032,14509.491,--,--,2852.6367,3978.038,4816.865,4.44162309305727e+38,1.60496095521651e+38,4.62137194701594e+38,1.3712964372218e+38,75712577634.1003,35711614246.0008,122.367513874161,54.5418378615914,3,3,270746099
1,HATLAS-SGP,HELP_J234053.127-315425.759,355.221364552832,-31.9071553050597,0.9664,--,--,--,20425.7,15119.818,8245.5205,--,--,4316.8525,4186.9756,5053.2725,3.08270302154601e+38,5.12118165487713e+37,3.17990970800791e+38,7.41496215632844e+37,143057050574.645,36965889609.7293,66.9553970372392,21.9321852841353,3,3,270745947
2,HATLAS-SGP,HELP_J234052.394-315358.753,355.218310013345,-31.8996535703626,0.6925,--,--,--,18308.998,10564.895,4270.3594,--,--,4194.416,4195.99,4371.3105,1.60490700911014e+38,4.06451271097613e+37,1.67679879370784e+38,4.57604902348417e+37,13595354035.8552,4676003980.64402,53.9513503646876,19.7897309863592,3,3,270745972
3,HATLAS-SGP,HELP_J234055.756-315329.992,355.232318025995,-31.8916643124239,0.5569,--,--,--,21086.463,26888.72,23736.895,--,--,4329.9004,4373.7285,4622.4805,1.50441064117579e+38,3.35024950533308e+37,1.52383234461298e+38,2.96113739958104e+37,24024594478.2071,15097055407.9407,42.7509671470982,14.2332299172373,3,3,270746634
4,HATLAS-SGP,HELP_J234100.230-315239.276,355.250959042836,-31.8775767794675,0.6144,--,--,--,37747.477,35127.543,21892.291,--,--,4500.0273,4949.3945,5417.959,2.58571861816433e+38,4.14934283911742e+37,2.77092791597091e+38,5.40726085559827e+37,7899208002.8411,3761643074.29722,118.097454641016,33.1457864044651,3,3,270746639
5,HATLAS-SGP,HELP_J234105.507-315427.894,355.272946715752,-31.9077483124058,0.5487,--,--,--,9599.586,5543.7695,13252.52,--,--,4341.37,3826.588,4615.673,6.57535978635814e+37,2.45138225548723e+37,7.0099005040519e+37,1.83900158056727e+37,7210830296.24201,3576156879.41248,24.1046788576206,9.96650291266059,3,3,270659244
6,HATLAS-SGP,HELP_J234109.718-315458.998,355.290492157837,-31.9163882312677,0.4713,--,--,--,7630.3643,8790.703,11874.207,--,--,4184.8584,4150.78,4781.6504,3.94061408126611e+37,2.51061484167469e+37,5.22554296314686e+37,1.50439169584799e+37,2076569476.0544,871581019.551319,16.9226437946661,13.9506758136917,3,3,270659237
7,HATLAS-SGP,HELP_J234106.293-315335.856,355.276221212023,-31.8932933250107,0.8565,--,--,--,35815.586,20942.742,15782.43,--,--,4419.373,4423.5684,5075.2695,3.70268506803201e+38,7.57982547867796e+37,4.31742880965391e+38,1.08700946861325e+38,19116882346.7954,7732443201.73702,131.619168758133,43.8724968271715,3,3,270659247
8,HATLAS-SGP,HELP_J234105.021-315241.042,355.270922427225,-31.8780671921819,0.8192,--,--,--,6632.401,8955.827,11774.765,--,--,4372.7563,4046.9668,5589.2354,1.22159227952387e+38,4.52160819845257e+37,1.11674776156302e+38,2.57050370801172e+37,38731931785.6575,15182525007.4566,30.6250750061685,14.8939130465412,3,3,270746642


In [6]:
def clean_table(table):
    """Take a table produced by a VO query and remove all empty columns
    
    Often many columns are empty and make the tables hard to read
    
    Inputs
    =======
    table
    
    Returns
    =======
    table
    
    """
    table = table.copy()
    for col in table.colnames:
        if np.all(table[col].mask):
            table.remove_column(col)
    for col in table.colnames:
        if table[col].dtype == 'object':
            print(col, table[col].dtype )
        
            table[col] = table[col].astype(str)
    return table
cigale_objects = clean_table(cigale_objects)

field object
help_id object


In [8]:
cigale_objects['field', 
    'help_id', 
    'ra', 
    'dec', 
    'redshift', 
    'zspec',
    'f_pacs_green',
    'ferr_pacs_green',
    'f_pacs_red',
    'ferr_pacs_red',
    'f_spire_250',
    'ferr_spire_250',
    'f_spire_350',
    'ferr_spire_350',
    'f_spire_500',
    'ferr_spire_500',
    'cigale_dustlumin',
    'cigale_dustlumin_err',
    'cigale_dustlumin_ironly',
    'cigale_dustlumin_ironly_err',
    'cigale_mstar',
    'cigale_mstar_err',
    'cigale_sfr',
    'cigale_sfr_err',
    'flag_optnir_obs',
    'flag_optnir_det',
    'hp_idx'].write('./data/cigale_objects.fits', overwrite=True)